In [2]:
key="7NvUEUX4tnzOja5KQ99gmUG37DQOV9oelvz1akWAr2Zts9X57djRMwbvfgjQoykp"
secret="X9CWCXNsdypjEU8Q0AQaoaqPrcnaX4wpDe5KVxsAfThkVJJAvufiGJ3tb95QqnQC"

# key api test2
import time



In [3]:
from binance_ft.um_futures import UMFutures
um_futures_client = UMFutures(key=key, secret=secret)
# um_futures_client = UMFutures(key=key, secret=secret, base_url="https://testnet.binancefuture.com")

from binance.spot import Spot
spot_client = Spot(api_key=key, api_secret=secret)

In [48]:

pair = "XAIUSDT"
pair_spot = pair.replace("USDT", "USDT")

future_info = um_futures_client.exchange_info()['symbols']
kk = [i['filters'] for i in future_info if i['symbol'] == pair][0][1]
print("future_info", kk)

print("spot_info", spot_client.exchange_info(pair_spot)['symbols'][0]['filters'][1])

bid_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])
ask_spot = float(spot_client.book_ticker(pair_spot)['askPrice'])

mark_price = float(spot_client.ticker_price(pair_spot)['price'])
mark_ft = float(um_futures_client.mark_price(pair)['markPrice'])
bid_spot, mark_price, ask_spot, mark_ft


future_info {'filterType': 'LOT_SIZE', 'minQty': '1', 'stepSize': '1', 'maxQty': '1000000'}
spot_info {'filterType': 'LOT_SIZE', 'minQty': '0.10000000', 'maxQty': '92141578.00000000', 'stepSize': '0.10000000'}


(0.9075, 0.9079, 0.9079, 0.9081)

In [84]:
# future truoc spot sau:
precision_ft = 0
precision_spot =1
usdt = 27
um_futures_client.change_leverage(symbol=pair, leverage=5, recvWindow=6000)

# print("order spot at", price_spot)


now_ft_price = float(um_futures_client.mark_price(pair)['markPrice']) 
price_ft = now_ft_price+0.0004
price_ft = round(price_ft, 4)

print("order ft at", price_ft)

quantity_ft = round((usdt)/now_ft_price, precision_ft)
open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="LIMIT", quantity=quantity_ft, price = price_ft, timeInForce="GTC")

# open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="MARKET", quantity=quantity_ft)
ft_order_id = open_future["orderId"]
print("ft_order_id =", ft_order_id)
while True:
    res_query_ft = um_futures_client.query_order(symbol=pair, orderId=ft_order_id, recvWindow=6000)
    if res_query_ft['status'] == "FILLED":
        # avg_spot = float(spot_order['price'])
        price_ft = float(res_query_ft['avgPrice'])
        executedQty_future = float(res_query_ft['executedQty'])
        usdt_value_ft = float(res_query_ft['cumQuote'])
        print("price_ft =", price_ft)
        print("usdt_value_ft =",usdt_value_ft)
        print("executedQty_future =", executedQty_future)
        print("type_ft =",res_query_ft['type'])
        print("---------------------")
        price_spot = float(spot_client.ticker_price(pair_spot)['price'])
        print("price_spot expect: ", price_spot)
        quantity_spot = round(usdt_value_ft/price_spot,precision_spot)
        open_spot = spot_client.new_order(symbol=pair_spot, side="BUY", type="LIMIT", quantity=quantity_spot, price=price_spot, timeInForce="GTC")
        order_id_spot = open_spot["orderId"]
        while True:
            res_query_spot = spot_client.get_order(symbol=pair_spot, orderId=order_id_spot, recvWindow=6000)
            if res_query_spot['status'] == "FILLED":
                executedQty_spot = float(res_query_spot['executedQty'])
                usdt_value_spot = float(res_query_spot['cummulativeQuoteQty'])
                price_spot = float(res_query_spot['price'])
                # print("spot:", price_spot, usdt_value_spot, "quantity:", executedQty_spot, res_query_spot['type'])
                print("price_spot =", price_spot)
                print("usdt_value_spot =",usdt_value_spot)
                print("executedQty_spot =", executedQty_spot)
                print("type_spot =",res_query_spot['type'])
                break
            time.sleep(0.1)
        break
    time.sleep(0.1)

order ft at 1.1079
ft_order_id = 101575969
price_ft = 1.109
usdt_value_ft = 26.616
executedQty_future = 24.0
type_ft = LIMIT
---------------------
price_spot expect:  1.1084
price_spot = 1.1084
usdt_value_spot = 26.5872
executedQty_spot = 24.0
type_spot = LIMIT


In [ ]:
# spot_client.cancel_open_orders(symbol=pair_spot)

In [85]:

fee_open = 0.00075 # use busd
fee_close = 0.00075

fee_ft_open = 0.00017
fee_ft_close = 0.00017
all_fee_per = fee_open + fee_close +fee_ft_open+fee_ft_close
all_fee_per



0.00184

In [86]:
fee_spot = (fee_open + fee_close)*usdt_value_spot
fee_future = (fee_ft_open + fee_ft_close) * usdt_value_ft
print("fee spot", fee_spot)
print("fee_future", fee_future)
while True:
    # curret_ft = float(um_futures_client.mark_price(pair)['markPrice'])
    # curret_spot = float(spot_client.ticker_price(pair_spot)['price'])

    curret_ft = float(um_futures_client.book_ticker(pair)['askPrice'])  # > mark price
    curret_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])  # < mark price

    gap_ft = -(curret_ft - price_ft)/price_ft * 100
    gap_spot = (curret_spot - price_spot)/price_spot * 100  

    if gap_spot + gap_ft > 0.25:
        print(gap_spot + gap_ft)

    pnl_ft = (price_ft - curret_ft)/price_ft * usdt_value_ft
    pnl_spot = (curret_spot-price_spot)/price_spot * usdt_value_spot








    
    # print("pnl ft: " ,pnl_ft, "pnl_spot", pnl_spot)


    print("pnl:",pnl_spot + pnl_ft - fee_spot - fee_future)
    if pnl_spot + pnl_ft - fee_spot - fee_future > usdt_value_spot * 0.0005 :
        # close position
        close_future =  um_futures_client.new_order(symbol=pair, side="BUY", type="LIMIT", quantity=executedQty_future, price = curret_ft, timeInForce="GTC")

        # close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
        close_spot = spot_client.new_order(symbol=pair_spot, side="SELL", type="LIMIT", quantity=executedQty_spot, price=curret_spot, timeInForce="GTC")
        close_future_id=close_future['orderId']
        close_spot_id=close_spot['orderId']

        
        # close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
        # close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)
        print("***************************** ", time.strftime("%Y-%m-%d %H:%M:%S"), " ****************")
        print("curret_ft", curret_ft)
        print("curret_spot", curret_spot)
        time.sleep(1)
        print("take order at:")
        time_done_spot = None
        time_done_ft = None

        while True:
            status_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future_id)
            status_spot_close = spot_client.get_order(symbol=pair_spot, orderId=close_spot_id, recvWindow=6000)
            if status_spot_close['status'] == "FILLED" and time_done_spot is None :
                time_done_spot = time.strftime("%Y-%m-%d %H:%M:%S")
            if len(status_ft_close) and time_done_ft is None:
                time_done_ft = time.strftime("%Y-%m-%d %H:%M:%S")
            if status_spot_close['status'] == "FILLED" and len(status_ft_close):
                price_ft_close_real = float(status_ft_close[0]['price'])
                # price_spot_close_real = float(close_spot['fills'][0]['price'])
                price_spot_close_real = curret_spot
                print("ft:", price_ft_close_real, time_done_ft )
                print("spot:", price_spot_close_real,time_done_spot )
                break
            else:
                time.sleep(1)

        break
    time.sleep(0.3)
    

fee spot 0.0398808
fee_future 0.00904944
pnl: -0.08734323169974849
pnl: -0.06814063335980113
pnl: -0.05853023999999894
pnl: -0.05133803501984816
pnl: -0.08254193252977428
pnl: -0.0537380350198479
pnl: -0.05133283833994921
pnl: -0.051331539169974474
pnl: -0.08730945328040529
pnl: -0.060905555770478646
pnl: -0.06809905992060417
pnl: -0.053697760750631006
pnl: -0.051299059920606
pnl: -0.06089256407073659
pnl: -0.05129386324071239
pnl: -0.05609776075063075
pnl: -0.04889646158066212
pnl: -0.04169516241068817
pnl: -0.04409516241068791
pnl: -0.04889516241068738
pnl: -0.04409516241068791
pnl: -0.039295162410688436
pnl: -0.03689516241068337
pnl: -0.051295162410687115
pnl: -0.051295162410687115
pnl: -0.06329386324071107
pnl: -0.03209126490075968
0.28853359817432367
pnl: 0.027873657509926417
*****************************  2024-01-16 00:02:34  ****************
curret_ft 1.1055
curret_spot 1.1081
take order at:
ft: 1.1055 2024-01-16 00:02:36
spot: 1.1081 2024-01-16 00:03:32


In [ ]:
# (curret_spot - price_spot)/price_spot +   -(curret_ft - price_ft)/price_ft
# (float(close_spot['fills'][0]['price']) - price_spot)/price_spot +   -(float(um_futures_client.get_account_trades(symbol=pair, orderId=close_future['orderId'])[0]['price']) - price_ft)/price_ft
# close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
# close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)


In [83]:
# calculate profit
BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])

# outcome_spot = sum([float(i['qty']) * float(i['price']) for i in close_spot['fills']]) #taker
outcome_spot = float(status_spot_close['cummulativeQuoteQty']) #maker

res_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future_id)
res_ft_open = um_futures_client.get_account_trades(symbol=pair, orderId=ft_order_id)

# assert float(res_ft_close["qty"]) == quantity_ft and float(res_ft_open["qty"]) == quantity_ft
# commision_close_spot = sum([float(i['commission']) for i in close_spot['fills']])*BNB_price
commision_close_spot = usdt_value_spot * 0.00075
commision_close_future = sum([float(res['commission']) for res in res_ft_close])*BNB_price
commision_open_future = sum([float(res['commission']) for res in res_ft_open])*BNB_price

pnl_ft =  sum([float(res['realizedPnl']) for res in res_ft_close])
pnl_spot = outcome_spot -  usdt_value_spot
all_profit = pnl_ft + pnl_spot - commision_close_future - commision_close_spot*2 - commision_open_future
all_profit


0.08981471200000031

In [ ]:
# response = um_futures_client.get_account_trades(symbol=pair,orderId=ft_order_id, recvWindow=6000)
# BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])
# float(response[0]['commission']) * BNB_price / usdt_value_ft

In [ ]:
commision_close_future / (0.03931*280)*100